In [1]:
!pip install tensorflow

^C


In [20]:
import numpy as np
import tensorflow as tf
from PIL import Image

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='best-int8 (2).tflite')
interpreter.allocate_tensors()

# Get input and output details.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the image.
image_path = 'img1.jpeg'
input_shape = input_details[0]['shape']
image = Image.open(image_path).resize((input_shape[1], input_shape[2]))
image_array = np.array(image)
image_array = image_array.astype(np.uint8)  # Convert image array to UINT8
input_data = np.expand_dims(image_array, axis=0)

# Set the input tensor to the image data.
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the inference.
interpreter.invoke()

# Get the output tensor and convert it to a numpy array.
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print the result.
print(output_data)

# Convert the output array to PIL Image
output_image = Image.fromarray(np.squeeze(output_data))

# Save the image to a file
output_image.save('output_image.JPG')

# Display the image
output_image.show()


[[[  2   2   4   4   0 241]
  [  2   2   4   5   0 241]
  [  2   2   4   4   0 241]
  ...
  [234 234  15  17   0 241]
  [232 230  21  26   0 241]
  [235 229  50  36   0 241]]]


In [23]:
import numpy as np
import tensorflow as tf
from PIL import Image

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='best-int8 (2).tflite')
interpreter.allocate_tensors()

# Get input and output details.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the image.
image_path = 'img4.jpg'

input_shape = input_details[0]['shape'][1:3]  # Excluding batch dimension
image = Image.open(image_path).resize(input_shape)
image_array = np.array(image) / 255.0  # Normalize pixel values between 0 and 1
input_data = np.expand_dims(image_array, axis=0).astype(input_details[0]['dtype'])

# Set the input tensor to the image data.
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the inference.
interpreter.invoke()

# Get the output tensor and convert it to a numpy array.
output_data = interpreter.get_tensor(output_details[0]['index'])

# Convert the output tensor to an image.
output_image = Image.fromarray(np.squeeze(output_data).astype(np.uint8))

# Print the image.
output_image.show()


In [25]:
import torch
import torchvision.transforms as transforms
import numpy as np
import tflite_runtime.interpreter as tflite

# Load the PyTorch model
model = torch.load('best.pt')
model.eval()

# Preprocess the input image
image_path = 'img4.jpg'
image = Image.open(image_path)
preprocess = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)

# Convert the PyTorch model to TensorFlow Lite
dummy_input = torch.randn(1, 3, 640, 640)
torch.onnx.export(model, dummy_input, 'model.onnx', verbose=True)
converter = tflite.TFLiteConverter.from_onnx('model.onnx')
tflite_model = converter.convert()

# Load the TensorFlow Lite model
interpreter = tflite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set the input tensor to the preprocessed image
input_tensor = np.array(input_batch, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_tensor)

# Run the inference
interpreter.invoke()

# Get the output tensor and post-process the results
output_tensor = interpreter.get_tensor(output_details[0]['index'])
output_classes = np.argmax(output_tensor, axis=1)

# Print the predicted class
class_labels = ['scartch/Damage']  # Replace with your own class labels
predicted_class = class_labels[output_classes[0]]
print("Predicted class:", predicted_class)


ImportError: generic_type: type "InterpreterWrapper" is already registered!

In [5]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

# Set the model to evaluation mode
model.eval()

# Load and preprocess the input image
image_path = '00.jpg'
image = Image.open(image_path)
preprocess = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])
input_tensor = preprocess(image)

# Perform object detection
results = model(input_tensor)

# Extract bounding box coordinates, confidences, and class labels
boxes = results.xyxy[0].tolist()
confidences = results.xyxy[0, :, 4].tolist()
class_ids = results.xyxy[0, :, 5].tolist()
labels = results.names[0]

# Display the detected objects
for box, confidence, class_id in zip(boxes, confidences, class_ids):
    x1, y1, x2, y2 = box
    label = labels[int(class_id)]
    print('Object:', label, 'Confidence:', confidence)
    print('Bounding Box:', (x1, y1), (x2, y2))
    print('---')


Using cache found in C:\Users\Anonymous Guy/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-1 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


ValueError: not enough values to unpack (expected 4, got 3)

In [8]:
import torch
import torchvision.transforms as transforms
import cv2

# Load the YOLOv5 model from a local folder
model_path = 'best.pt'
model = torch.load(model_path)  # Use map_location to load on CPU if trained on GPU
model = checkpoint['model']
# Set the model to evaluation mode
model.eval()

# Load the image
image_path = "image.jpg"
image = cv2.imread(image_path)

# Preprocess the image
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])
input_tensor = preprocess(image).unsqueeze(0)

# Detect objects
with torch.no_grad():
    output = model(input_tensor)

# Extract bounding box coordinates, confidences, and class labels
boxes = output.xyxy[0].tolist()
confidences = output.xyxy[0, :, 4].tolist()
class_ids = output.xyxy[0, :, 5].tolist()
labels = output.names[0]

# Draw bounding boxes on the image
for box, confidence, class_id in zip(boxes, confidences, class_ids):
    x1, y1, x2, y2 = box
    label = labels[int(class_id)]
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    cv2.putText(image, label, (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image
cv2.imshow("Image with detected objects", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


NameError: name 'checkpoint' is not defined

!streamlit hello

In [8]:
!streamlit run detector_made.py

^C


In [ ]:
import os 
os.system("python detect.py --weights \"best.pt\" --img 640 --conf 0.25 --source \"image_find.jpg\" > output.txt 2>&1")


0

In [22]:
filename = "output.txt"
import re
# Read the content of the file
with open(filename, "r") as file:
    # Iterate over each line in the file
    for line in file:
        # Print each line
        print(line.strip())
filename = "output.txt"
extracted_string=""
# Read the content of the file
with open(filename, "r") as file:
    # Iterate over each line in the file
    for line in file:
        match = re.search(r'runs\\detect\\(exp\d+)\.', line)
        if match:
              extracted_string = match.group(1)
        # Print each line
        #print(line.strip())
print(extracted_string)



detect: weights=['best.pt'], source=image_find.jpg, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to 'C:\Users\Anonymous': Invalid argument
YOLOv5  2023-5-28 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers...
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 C:\Users\Anonymous Guy\Desktop\yolov5\image_find.jpg: 640x640 (no detections), 14.0ms
Speed: 1.0ms pre-process, 14.0ms inference, 0.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp4

